<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### 0. Introduction
This Notebook is for Coursera course, Applied Data Science Capstone, week 3 assignment.

In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### 1. Collect and collate data from Wikipedia

In [4]:
#read and copy the website data
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wb_data = requests.get(url)
soup = BeautifulSoup(wb_data.text,'lxml')

#find the content corresponding to the table
table = soup.find('table')

#get the table headers col_names
headers = table.find_all('th')
col_names = []
for i in range(len(headers)):
    col_names.append(headers[i].text.rstrip())
print('There are ', len(col_names), ' columns in the table, headers are ')
print(col_names)

#get the detailed table contents
contents = []
for row in table.find_all('td'):
    contents.append(row.text.rstrip())
print('\nThere are ', int(len(contents)/3), ' rows in the table.')

There are  3  columns in the table, headers are 
['Postal code', 'Borough', 'Neighborhood']

There are  180  rows in the table.


##### Structure the table contents in a dataframe
1. Ignore cells with a borough that is 'Not assigned'.
2. Neighborhoods sharing the same postal code will be combined into one row with a comma for seperation.
3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
#raw table from Wikipedia
df_raw = pd.DataFrame(columns = col_names, index=range(180))
for i in range(int(len(contents)/3)):
    df_raw['Postal code'][i] = contents[3*i + 0]
    df_raw['Borough'][i] = contents[3*i + 1]
    df_raw['Neighborhood'][i] = contents[3*i + 2]

#remove 'Not assigned'
df = df_raw[df_raw.Borough !='Not assigned']

#seperate with comma
df.reset_index(drop= True, inplace=True)

for i in range(df.shape[0]):
    if "/" in df['Neighborhood'][i]:
        df['Neighborhood'][i] = df['Neighborhood'][i].replace(" / ", ', ')

df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
#print the number of rows of the dataframe
df.shape

(103, 3)

<h1><font size = 3>This is the end of <span style="color:red">Assignment Part1</span>: create the required dataframe and print its number of rows in last cell</font></h1>
<h1><font size = 3>Now, let's start<span style="color:red"> Assignment Part2</span>: create the dataframe with latitude and longtigude</font></h1>

In [36]:
#import the csv file that has the geographical coordinates of each postal code
data = pd.read_csv('~\Desktop\python\Geospatial_Coordinates.csv')
coords = pd.DataFrame(data)
coords.rename(columns={'Postal Code': 'Postal code'}, inplace=True)

#merge the latitude and longitude values from coords to df
Toronto_df = left_merge = pd.merge(df, coords, on='Postal code', how='left')
Toronto_df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1><font size = 3>This is the end of <span style="color:red">Assignment Part2</span>: ccreate the dataframe with latitude and longtigude</font></h1>
<h1><font size = 3>Now, let's start<span style="color:red"> Assignment Part3</span>: Explore and cluster the neighborhoods in Toronto</font></h1>

In [37]:
import json
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim

##### Use geopy library to get the latitude and longitude values of Toronto.

In [38]:
address = 'Toronto'
geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [49]:
credentials = pd.read_csv('~\Desktop\python\credentials.csv')
CLIENT_ID = credentials['CLIENT_ID'][0] # my Foursquare ID
CLIENT_SECRET = credentials['CLIENT_SECRET'][0] # my Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

##### Explore Neighborhoods in Toronto

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Let's create a function to explore all the boroughs in Toronto

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each borough and create a new dataframe called *toronto_venues*.

In [63]:
toronto_venues = getNearbyVenues(names=Toronto_df['Borough'],latitudes=Toronto_df['Latitude'],longitudes=Toronto_df['Longitude'])

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [64]:
#check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2147, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,North York,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [65]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 uniques categories.


##### Analyze Each Borough


In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#group rows by borough and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()

toronto_grouped

,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026549,0.000000,...,0.000000,0.008850,0.000000,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.008850
1,Downtown Toronto,0.000000,0.000814,0.000814,0.000814,0.001629,0.002443,0.001629,0.015472,0.001629,...,0.002443,0.010586,0.001629,0.000000,0.003257,0.000000,0.004886,0.000000,0.000814,0.005700
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023438,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007812,0.000000,0.000000,0.023438
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.012658,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004132,0.004132,0.000000,0.000000,0.000000,0.000000,0.000000,0.008264,0.000000,...,0.000000,0.000000,0.004132,0.004132,0.008264,0.000000,0.000000,0.000000,0.008264,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,...,0.010638,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.019481,0.000000,0.000000,0.012987,0.000000,0.006494,0.000000,0.000000,0.012987
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000


#### Clustering

In [68]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1])

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [75]:
#create a new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Coffee Shop,Clothing Store,Restaurant,Pizza Place,Bank,Japanese Restaurant,Grocery Store,Park,Sandwich Place,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Clothing Store,Restaurant,Pizza Place,Bank,Japanese Restaurant,Grocery Store,Park,Sandwich Place,Fast Food Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Seafood Restaurant,Park,American Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Coffee Shop,Clothing Store,Restaurant,Pizza Place,Bank,Japanese Restaurant,Grocery Store,Park,Sandwich Place,Fast Food Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Seafood Restaurant,Park,American Restaurant


In [78]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters